# MIRI LRS Spectral Extraction Techniques

**Use case:** Extract spectra with different techniques.<br>
**Data:** MIRI LRS spectrum of Type Ia supernova SN2021aefx, observed by Jha et al in PID 2072 (Obs 1), **where the automated spectral extraction failed**. These data were taken with zero exclusive access period, and published in [Kwok et al 2023](https://ui.adsabs.harvard.edu/abs/2023ApJ...944L...3K/abstract).<br> 
**Tools:** jdaviz, specviz2d, specreduce, jwst, gwcs, matplotlib, astropy.<br>
**Cross-intrument:** NIRSpec, MIRI.<br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

# Introduction

This notebook extracts a 1D spectra from a 2D MIRI LRS spectral observation (single image).  The goal is to provide the ability to extract spectra with different locations, extraction apertures, and techniques than are done in the JWST pipeline using the [Astropy Specreduce package](https://github.com/astropy/specreduce).

The notebook also demos how to use Jdaviz's [specviz2d](https://jdaviz.readthedocs.io/en/latest/specviz2d/index.html), which allows users to interactively extract 1D spectra from 2D spectra.

The simpliest spectral extraction is "boxcar" where all the pixels within some fixed width centered on the source position are summed at each wavelength.  Background subtraction can be done using regions offset from the source center. You can also see the Specreduce [generic Sample Notebook](https://github.com/astropy/specreduce/blob/main/notebook_sandbox/jwst_boxcar/boxcar_extraction.ipynb).

For spectra taken with a diffraction limited telescope like JWST, a modification boxcar extraction is to vary the extraction width linearly with wavelength.  Such a scaled boxcar extraction keeps the fraction of the source flux within the extraction region approximately constant with wavelength.

For point sources, a PSF-weighted spectral extraction can be done.  Using the PSF to weight the extraction uses the actual PSF as a function of wavelength to optimize the extraction to the pixels with the greatest signal.  PSF-weighted extractions show the largest differences with boxcar extractions at lower S/N values.

**Note:** Corrections for the finite aperture used in all the extractions have not been applied.  Thus, the physical flux densities of all the extracted spectra are lower than the actual values.

# Imports

- *matplotlib.pyplot* for plotting data
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *astropy.visualization* for scaling image for display
- *astropy.table Table* for reading the pipeline 1d extractions
- *jwst datamodels* for reading/access the jwst data

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> Begin PEP8 check cells (delete below when finished)</b></p>

<p style="font-size:200%; color:#e56020; background-color:#1d1160;"><b><i>Reviewer note:</i> Begin PEP8 check cells (delete above when finished)</b></p>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
# %matplotlib inline

import numpy as np

from gwcs.wcstools import grid_from_bounding_box

from astropy.io import fits
from astropy.table import Table
from astropy.visualization import simple_norm

from jwst import datamodels

from specreduce.extract import BoxcarExtract, OptimalExtract, HorneExtract
from specreduce.tracing import FlatTrace, FitTrace
from specreduce.background import Background

from jdaviz import Imviz
from jdaviz import Specviz
from jdaviz import Specviz2d

from astropy.utils.data import download_file
import os

from specutils import Spectrum1D
from astropy import units as u

# Display the video
from IPython.display import HTML, YouTubeVideo

import os
import urllib.request
import tarfile

In [ ]:
# Download all necessary data

if os.path.exists("boxcar_specviz2d.fits"):
    print("Boxcar Specviz2d Extraction Exists")
else:
    url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/boxcar_specviz2d.fits'
    urllib.request.urlretrieve(url)

if os.path.exists("horne_specviz2d.fits"):
    print("Horne Specviz2d Extraction Exists")
else:
    url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/horne_specviz2d.fits'
    urllib.request.urlretrieve(url)

In [ ]:
if os.path.exists("./data/"):
    print("Origina Data Exists")
else:
    url = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/data.tar.gz'
    urllib.request.urlretrieve(url)
    
    # Unzip files if they haven't already been unzipped
    if os.path.exists("data/"):
        print("Data Directory Already Exists")
    else:
        tar = tarfile.open('./data.tar.gz', "r:gz")
        tar.extractall()
        tar.close()

#### Developer note: Ask Karl exactly how these functions work? Seems like all weights are equal?

In [ ]:
# useful functions
def get_boxcar_weights(center, hwidth, npix):
    """
    Compute the weights given an aperture center, half widths, and number of pixels
    """
    weights = np.zeros((npix))
    # pixels with full weight
    fullpixels = [max(0, int(center - hwidth + 1)), min(int(center + hwidth), npix)]
    weights[fullpixels[0] : fullpixels[1]] = 1.0

    # pixels at the edges of the boxcar with partial weight
    if fullpixels[0] > 0:
        weights[fullpixels[0] - 1] = hwidth - (center - fullpixels[0])
    if fullpixels[1] < npix:
        weights[fullpixels[1]] = hwidth - (fullpixels[1] - center)

    return weights


def ap_weight_images(
    center, width, bkg_offset, bkg_width, image_size, waves, wavescale=None
):
    """
    Create a weight image that defines the desired extraction aperture
    and the weight image for the requested background regions

    Parameters
    ----------
    center : float
        center of aperture in pixels
    width : float
        width of apeture in pixels
    bkg_offset : float
        offset from the extaction edge for the background
        never scaled for wavelength
    bkg_width : float
        width of background region
        never scaled with wavelength
    image_size : tuple with 2 elements
        size of image
    waves : array
        wavelegth values
    wavescale : float
        scale the width with wavelength (default=None)
        wavescale gives the reference wavelenth for the width value

    Returns
    -------
    wimage, bkg_wimage : (2D image, 2D image)
        wimage is the weight image defining the aperature
        bkg_image is the weight image defining the background regions
    """
    wimage = np.zeros(image_size)
    bkg_wimage = np.zeros(image_size)
    hwidth = 0.5 * width
    # loop in dispersion direction and compute weights
    for i in range(image_size[1]):
        if wavescale is not None:
            hwidth = 0.5 * width * (waves[i] / wavescale)

        wimage[:, i] = get_boxcar_weights(center, hwidth, image_size[0])

        # bkg regions
        if (bkg_width is not None) & (bkg_offset is not None):
            bkg_wimage[:, i] = get_boxcar_weights(
                center - hwidth - bkg_offset, bkg_width, image_size[0]
            )
            bkg_wimage[:, i] += get_boxcar_weights(
                center + hwidth + bkg_offset, bkg_width, image_size[0]
            )
        else:
            bkg_wimage = None

    return (wimage, bkg_wimage)

### Devloper notes (2021)

1) The difference between the pipeline (x1d) and the extractions done in this notebook are quite large.  Help in understanding the origin of these differences is needed.

2) Not clear how to use the JWST pipeline `extract_1d` (quite complex) code. Help to determine how to use the JWST pipeline code instead of the custom code for boxcar is needed. 

3) Applying aperture corrections for the finite extraction widths is needed.  Help in how to get the needed informatinom for different (user set) extraction widths is needed. 

### Partially RESOLVED (March, 2023)

1) See notes from Kendrew on limitations of current pipeline. Pipeline will be updated soon.

2) While this notebook doesn't go into using the pipeline, boxcar is now integrated into the Astropy Specreduce package. So I wouldn't characterize the boxcar as "custom code" any longer.

3) Still not resolved.

## Download Files

In [ ]:
#calfilename = "det_image_seq5_MIRIMAGE_P750Lexp1_cal.fits"
s2dfile = "./data/jw02072-o001_t010_miri_p750l_s2d.fits"
x1dfile = "./data/jw02072-o001_t010_miri_p750l_x1d.fits"
spatialprofilefile = "./data/jw02072-o001_t010_miri_p750l_s2d.fits"
#mainurl = "https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/"

#calfile_dld = download_file(mainurl + calfilename)
#s2dfile_dld = download_file(mainurl + s2dfilename)
#x1dfile_dld = download_file(mainurl + x1dfilename)
#spatialprofilefile_dld = download_file(mainurl + spatialprofilefilename)

## File information

The data used is a simulation of a LRS slit observation for a blackbody with a similar flux density to the star BD+60d1753, a flux calibration star.  This simulation was created with MIRISim.
The simulated exposure was reduced using the JWST pipeline (v0.16.1) through the Detector1 and Spec2 stages.

The cal file is one of the Spec2 products and is the calibration full frame image. It contains:

1. (Primary): This HDU contains meta-data related to the observation and data reduction.
2. (SCI): The calibrated image. Units are MJy/sr.
3. (ERR): Uncertainty image.  Units are MJy/sr.
4. (DQ): Data quality image.
5. (VAR_POISSON): Unc. component 1: Poisson uncertainty image.  Units are (MJy/sr)^2.
6. (VAR_RNOISE): Unc. component 2: Read Noise uncertainty image.  Units are (MJy/sr)^2.
7. (VAR_FLAT): Unc. component 3: Flat Field uncertainty image.  Units are (MJy/sr)^2.
8. (ASDF_METADATA): Metadata.

The s2d file is one of the Spec2 products and containes the calibrated rectified cutout of the LRS Slit region.  It has:

1. (Primary): This HDU contains meta-data related to the observation and data reduction.
2. (WGT): Weight.
3. (CON): ??
4. (ASDF_METADATA): Metadata.

## Loading data

In [ ]:
# use a jwst datamodel to provide a good interface to the data and wcs info
#cal = datamodels.open(calfile)
s2d = datamodels.open(s2dfile)

Basic information about the image.

In [ ]:
#print("cal image")
#print(cal.data.shape)
#print(np.mean(cal.data))
#print(np.amin(cal.data), np.amax(cal.data))
print("s2d image")
print(s2d.data.shape)
print(np.mean(s2d.data))
print(np.amin(s2d.data), np.amax(s2d.data))

Display the full 2D image

Display the LRS Slit region only (use s2d)

In [ ]:
# transpose to make it display better
image = np.transpose(s2d.data)
err = np.transpose(s2d.err)
norm_data = simple_norm(image, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(image, norm=norm_data, origin="lower")
plt.title("The LRS region")

#### View the 2D Spectrum in Imviz and get the center of the cross-dispersion 

In [ ]:
imviz = Imviz()
imviz.app

In [ ]:
imviz.load_data(image)

In [ ]:
viewer = imviz.default_viewer
viewer.cuts = '95%'

# 1) Default JWST Pipeline 1D extraction

In [ ]:
# Create a spectrum1d
jpipe_x1d = Spectrum1D.read(x1dfile)

In [ ]:
specviz = Specviz()
specviz.app

In [ ]:
specviz.load_spectrum(jpipe_x1d)

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend()

# 2) Fixed Width Boxcar Extraction (Using Specreduce)

Extract a 1D spectrum using a simple boxcar.  Basically collapse the spectrum in the cross-dispersion direction over a specified number of pixels.

#### Developer note: Allow for a bad pixel mask

## Define extraction parameters

In [ ]:
ext_center = 31.5
ext_width = 8
bkg_sep = 7
bkg_width = 2

Plot cross-disperion cut showing the extraction parameters

#### Develepor Note: Place trace back into Specviz2d/Imviz/Etc

In [ ]:
# Plot along cross-disperion cut showing the extraction parameters
fig, ax = plt.subplots(figsize=(10, 6))
y = np.arange(image.shape[0])
ax.plot(y, image[:,300], 'k-')
mm = np.array([ext_center, ext_center])
mm_y = ax.get_ylim()

# extraction region
ax.axvspan(ext_center - ext_width/2., ext_center + ext_width/2., color='green', alpha=0.1)
ax.plot(mm, mm_y, 'b--')
ax.plot(mm - ext_width/2., mm_y, 'g:')
ax.plot(mm + ext_width/2., mm_y, 'g:')

# background region, symmetric on both sides of extraction region
ax.axvspan(ext_center - bkg_sep - bkg_width/2., ext_center - bkg_sep + bkg_width/2., color='red', alpha=0.1)
ax.plot(mm - bkg_sep - bkg_width/2., mm_y, 'r:')
ax.plot(mm - bkg_sep + bkg_width/2., mm_y, 'r:')

ax.axvspan(ext_center + bkg_sep - bkg_width/2., ext_center + bkg_sep + bkg_width/2., color='red', alpha=0.1)
ax.plot(mm + bkg_sep - bkg_width/2., mm_y, 'r:')
ax.plot(mm + bkg_sep + bkg_width/2., mm_y, 'r:')

ax.set_title("Cross-dispersion Cut at Pixel=300")

## Define Background

In [ ]:
# extract the background using custom individual traces
trace = FlatTrace(image, ext_center)
bg = Background(image, [trace-bkg_sep, trace+bkg_sep], width=bkg_width)

# alternatively, call two_sided class, which does the same as above 
#bg = Background.two_sided(image, trace, bkg_sep, width=bkg_width)
# or in the place of any trace, an int/float can be passed which resolves to a FlatTrace
#bg = Background.two_sided(image, ext_center, bkg_sep, width=bkg_width)

# or for single sided:
# bg = Background.one_sided(image, trace, bkg_sep, width=bkg_width)
# bg = Background.one_sided(image, trace, -bkg_sep, width=bkg_width)

In [ ]:
# view the background weighted image
plt.figure(figsize=(15, 15))
plt.imshow(bg.bkg_wimage, origin="lower")
plt.title("slit[0] slice")

In [ ]:
# view the background image
plt.figure(figsize=(15, 15))
plt.imshow(bg.bkg_image().flux.value, norm=norm_data, origin="lower")
plt.title("slit[0] slice")

In [ ]:
# view the background-subtracted image
plt.figure(figsize=(15, 15))
plt.imshow(bg.sub_image().flux.value, norm=norm_data, origin="lower")
plt.title("slit[0] slice")

Note that when using median to calculate the background, partial pixel weights are not supported:

In [ ]:
bg_med = Background.two_sided(image, ext_center, bkg_sep, width=bkg_width, statistic='median')
plt.figure(figsize=(15, 15))
plt.imshow(bg_med.bkg_wimage, origin="lower")
plt.title("slit[0] slice")

## Extract Trace (multiple options)

Optional: we could now refine the initial flat trace by running an automated FitTrace on the subtracted image. This process could be iterated as necessary (recreating the subtracted image with the refined trace, etc).

In [ ]:
fit_trace = FitTrace(image-bg.bkg_image().flux.value, peak_method='gaussian', guess=ext_center)
#fit_trace

In [ ]:
flat_trace = FlatTrace(image-bg.bkg_image().flux.value, trace_pos=ext_center)
#flat_trace

#### <span style="color:red">**Always visualize your traces. If you have noisy data, the fits may not be good. You may need to play around with the type of fit (i.e., Order 1 Polynomial) or different window sizes and parameters. In our case, we'll stick with the flat trace throughout this notebook.** 
</span>

#### Plot old vs new trace

In [ ]:
fig3, ax3 = plt.subplots(figsize=(10,6))
plot3 = ax3.imshow(bg.sub_image().flux.value, aspect=4.,
                   vmin=0, vmax=bg.sub_image().flux.value.max()/2,
                   cmap=mpl.cm.magma, origin='lower',
                   extent=(0, bg.sub_image().flux.value.shape[-1],
                           0, bg.sub_image().flux.value.shape[0]))
fig3.colorbar(plot3)
ax3.set_title('LRS Spectrum Traces')
ax3.grid()

# add the traces
ax3.plot(flat_trace.trace, '--', color='#008ca8',
         lw=2.5, label='FlatTrace')
ax3.plot(fit_trace.trace, '--', color='#00471b',
         lw=2.5, label='GaussianFitTrace')
ax3.legend(framealpha=.5)
plt.show()

## Extract

#### Dev Note: FitTrace doesn't seem to be working right now, so we'll stick with FlatTrace

In [ ]:
#boxcar = BoxcarExtract()
ext1d_boxcar_noweights = BoxcarExtract(image-bg, flat_trace, width=ext_width)
ext1d_boxcar_noweights = ext1d_boxcar_noweights.spectrum.flux.value
ext1d_boxcar_noweights *= 1e6 * s2d.meta.photometry.pixelarea_steradians
#spectrum_specreduce_boxcar

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))

ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")
ax.plot(jpipe_x1d.spectral_axis, ext1d_boxcar_noweights, color = 'blue', label='Boxcar')
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend()

# 3) Fixed Width Boxcar Extraction (Using Pixel Masks, too)


#### <span style="color:red">**A basic example of how to create a pixel weight map/mask. In this example, the weights basically create a pixel mask based on the boxcar extraction aperture. It shouldn't actually change any of the results because the boxcar extraction essentially does this for you. But this provides a useful example for more complicated masks that we will create lower in the notebook.**</span> 

In [ ]:
from gwcs.wcstools import grid_from_bounding_box

#image = np.transpose(s2d.data)
grid = grid_from_bounding_box(s2d.meta.wcs.bounding_box)
ra, dec, lam = s2d.meta.wcs(*grid)
lam_image = np.transpose(lam)

# compute a "rough" wavelength scale to allow for aperture to scale with wavelength
rough_waves = np.average(lam_image, axis=0)

# images to use for extraction
wimage, bkg_wimage = ap_weight_images(
    ext_center,
    ext_width,
    bkg_width,
    bkg_sep,
    image.shape,
    rough_waves,
    wavescale=None,
)

#boxcar = BoxcarExtract()

# without background subtraction
image_wg = image * wimage
ext1d_boxcar = BoxcarExtract(image_wg, flat_trace, width=ext_width)
ext1d_boxcar = ext1d_boxcar.spectrum.flux.value
# convert from MJy/sr to Jy
ext1d_boxcar *= 1e6 * s2d.meta.photometry.pixelarea_steradians

# with background subtraction
image_bg = bg.sub_image()
image_wg = image_bg * wimage
ext1d_boxcar_bkgsub = BoxcarExtract(image_wg, flat_trace, width=ext_width)
ext1d_boxcar_bkgsub = ext1d_boxcar_bkgsub.spectrum.flux.value

# convert from MJy/sr to Jy
ext1d_boxcar_bkgsub *= 1e6 * s2d.meta.photometry.pixelarea_steradians

# compute the average wavelength for each column using the weight image
# this should correspond directly with the extracted spectrum
# wavelengths account for any tiled spectra this way
waves_boxcar = np.average(lam_image, weights=wimage, axis=0)
waves_boxcar_bkgsub = np.average(lam_image, weights=wimage, axis=0)

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar_bkgsub > 0.
gpts = ext1d_boxcar > 0.

ax.plot(jpipe_x1d.spectral_axis, ext1d_boxcar_noweights, color = 'blue', label = 'Boxcar; No Mask')
ax.plot(waves_boxcar[gpts], ext1d_boxcar[gpts], color = 'green', label="Boxcar; Mask")
ax.plot(waves_boxcar_bkgsub[gpts], ext1d_boxcar_bkgsub[gpts], color = 'red', label="Boxcar; bkgsub; Mask")
ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")
ax.set_title("Fixed boxcar 1D extracted spectrum")
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend()

# 4) Wavelength scaled width boxcar

The LRS spatial profile changes as a function of wavelength as JWST is diffraction limited at these wavelengths.  Nominally this means that the FWHM is changing linearly with wavelength.  Scaling the width of the extraction aperture with wavelength accounts for the changing diffraction limit with wavelength to first order.

#### Developer note:  Not currently possible. Allow for wavelength scaled width in the boxcar

# 5) Horne Extraction

#### <span style="color:red">**The Horne algorithm preforms a Gaussian fit on the source, it is thus best suited for cases where the source has a Gaussian profile in the cross-dispersion direction. If your profile is not Gaussian, you will likely over- or under-estimate your actual flux.**. 
</span>


### Steps in the original Horne (1986) paper:

1. Bias subtraction [assumed to be done in earlier block]
2. Initial variance estimate [user provides this as an argument]
3. Fit sky background [assumed to be done in earlier block]
 * "We therefore generally perform a least-squares polynomial fit to the sky data at each wavelength. Individual sky pixels are given weights inversely proportional to their variances as estimated in Step 2" [overlaps with notebook guide's 3b]
4. Extract standard spectrum and its variance
 * Subtract the sky background found in Step 3 from the image. [sky background calculation is planned as a separate, earlier step of the specreduce workflow]
5. Construct spatial profile
6. Revise variance estimates [not currently done]
7. Mask cosmic ray hits [not currently done]
8. Extract optimal spectrum and its variance [currently only extract the spectum, not a variance]
9. Iterate Steps 5-8

The first four steps as the standard procedure and the last five as add-ons that help squeeze out extra signal-to-noise.

There are notes [in brackets] on how each step is handled in the proposed HorneExtract/OptimalExtract classes to make it easier to see what the class does and what the user must do themselves.

### Steps in the JDAT Notebook guide on optimal extraction:

1. Define extraction region [user's responsibility to provide an appropriate image]
2. Pick a slice [should not be necessary? can use the whole image as the aperture with good results]
3. Define extraction kernel
 * Select PSF template [assumed to be Gaussian for now. support for Moffat, others?]
 * Choose a polynomial for background fitting [user provides as an argument]
4. Fit extraction kernel to initial slice [all columns are coadded to perform the fit]
5. Fit geometric distortion [not currently done]
 * Determine cross-dispersion bins for trace fitting
 * Fit a kernel to each bin to find trace center [user provides this as a specreduce.tracing.Trace object]
 * Polynomial fit of trace centers
6. Combine composite model with 2D image to create output 1D spectrum
 * Create variance image [user provides this as an argument]
 * Generate 1D spectrum
7. Compare with reference 1D spectrum

In [ ]:
ext1d_horne = HorneExtract(image-bg, flat_trace)
ext1d_horne = ext1d_horne.spectrum.flux.value
# convert from MJy/sr to Jy
ext1d_horne *= 1e6 * s2d.meta.photometry.pixelarea_steradians

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar_bkgsub > 0.
gpts = ext1d_boxcar > 0.

ax.plot(jpipe_x1d.spectral_axis, ext1d_boxcar_noweights, color = 'blue', label = 'Boxcar; No Mask')
ax.plot(jpipe_x1d.spectral_axis, ext1d_horne, color = 'purple', label = 'Horne; No Mask')
ax.plot(waves_boxcar[gpts], ext1d_boxcar[gpts], color = 'green', label="Boxcar; Mask")
ax.plot(waves_boxcar_bkgsub[gpts], ext1d_boxcar_bkgsub[gpts], color = 'red', label="Boxcar; bkgsub; Mask")
ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")
ax.set_title("Fixed boxcar 1D extracted spectrum")
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend()

#### <span style="color:red">**See note above. In this case the Horne extraction likely overestimates the flux because it under-fits the wings of the cross-dispersion profile. Using a real MIRI LRS PSF is a better idea.**</span> 


# 5) PSF-Weighted Extraction

While to first order the PSF FHWM changes linearly with wavelength, this is an approximation.  It is better to use the measured spatial profile as a function of wavelength to extract the spectrum.  This tracks the actual variation with wavelength and optimizes the extraction to the higher S/N measurements.  In general, PSF based extractions show the most improvements over boxcar extractions at lower the S/N.

There are two PSF based extraction methods:

1. PSF weighted: the spatial profile at each wavelength is used to weight the extraction.
2. PSF fitting: the spatial profile is fit at each wavelength with the scale parameter versus wavelength giving the spectrum.

#### Only the PSF weighted technique is currently part of this notebook.

Note 1: calibration reference file for the specific LRS slit position should be used.<br>
Note 2: Small shifts in the centering of the source in the slit should be investigated to see if they impact the PSF based extractions.<br>
Limitation: currently it is assumed there are no bad pixels.<br>

### PSF weighted extaction

#### Generate the PSF profile as a function of wavelength
For MIRI LRS slit observations, observations are made at two nod position in the slit after target acquisition.  This means that the location of the sources in the slit is very well known.  Hence, spatial profile (PSF) as a function of wavelength for the two nod positions is straightforward to measure using observations of a bright source.

The next few steps generate the needed information for the nod position for which we are extracting spectra based on a simulation of a bright source at the same nod position.

In [ ]:
# lrs spatial profile (PSF) as a function of wavelength
# currently, this is just a "high" S/N observation of a flat spectrum source at the same slit position
psf = datamodels.open(spatialprofilefile)
# transpose to make it display better
lrspsf = np.transpose(psf.data)
norm_data = simple_norm(lrspsf, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(lrspsf, norm=norm_data, origin="lower")
plt.title("The LRS Spatial Profile (PSF) Observation")

In [ ]:
# Mock a LRS spectral profile reference file
# Sum along the spatial direction and normalize to 1
# assume there is no background (none was included in the MIRISim for the flat spectrum source observation)
# ignore regions far from the source using a scaled boxcar weight image
# the aperture (psf_width) used in the scaled boxcar weight image could be varied
psf_width = 12.0
(wimage_scaledboxcar, tmpvar) = ap_weight_images(ext_center, psf_width, bkg_sep, 
                                                 bkg_width, image.shape, waves_boxcar, wavescale=10.0)

psf_weightimage = lrspsf*wimage_scaledboxcar

# generate a 2D image of the column sums for division
max_psf = np.max(psf_weightimage, axis=0)
div_image = np.tile(max_psf, (psf_weightimage.shape[0], 1))
div_image[div_image == 0.0] = 1.0  # avoid divide by zero issues

# normalize 
psf_weightimage /= div_image

# display
norm_data = simple_norm(psf_weightimage, "sqrt")
plt.figure(figsize=(10, 3))
plt.imshow(psf_weightimage, norm=norm_data, origin="lower")
plt.title("The LRS Spatial Profile Reference Image (Normalized)")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
y = np.arange(psf_weightimage.shape[0])
ax.plot(y, psf_weightimage[:,150], label="pixel=150")
ax.plot(y, psf_weightimage[:,225], label="pixel=225")
ax.plot(y, psf_weightimage[:,300], label="pixel=300")
ax.plot(y, psf_weightimage[:,370], label="pixel=370")
ax.set_title("Cross-dispersion Cuts")
ax.set_xlim(ext_center-psf_width, ext_center+psf_width)
ax.legend()

Note that the spatial profile becomes narrower as the pixel values increases as this corresponds to the wavelength decreasing.

#### Extract spectrum using wavelength dependent PSF profiles using the same traces as defined above

In [ ]:
image_bg = bg.sub_image()
image_wg = image_bg * psf_weightimage
ext1d_boxcar_bkgsub_psfweight = BoxcarExtract(image_wg, flat_trace, width=ext_width)
ext1d_boxcar_bkgsub_psfweight = ext1d_boxcar_bkgsub_psfweight.spectrum.flux.value

# convert from MJy/sr to Jy
ext1d_boxcar_bkgsub_psfweight *= 1e6 * s2d.meta.photometry.pixelarea_steradians

# compute the average wavelength for each column using the weight image
# this should correspond directly with the extracted spectrum
# wavelengths account for any tiled spectra this way
waves_boxcar_psfweight = np.average(lam_image, weights=wimage, axis=0)
waves_boxcar_bkgsub_psfweight = np.average(lam_image, weights=wimage, axis=0)

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar_bkgsub > 0.
gpts = ext1d_boxcar > 0.

ax.plot(jpipe_x1d.spectral_axis, ext1d_boxcar_noweights, color = 'blue', label = 'Boxcar; No Mask')
ax.plot(jpipe_x1d.spectral_axis, ext1d_horne, color = 'purple', label = 'Horne; No Mask')
ax.plot(waves_boxcar[gpts], ext1d_boxcar[gpts], color = 'green', label="Boxcar; Mask")
ax.plot(waves_boxcar_bkgsub[gpts], ext1d_boxcar_bkgsub[gpts], color = 'red', label="Boxcar; bkgsub; Mask")
ax.plot(waves_boxcar_bkgsub_psfweight[gpts], ext1d_boxcar_bkgsub_psfweight[gpts], 'k-', label="Boxcar; bkgsub; PSF Weights", color='cyan')
ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")
ax.set_title("Fixed boxcar 1D extracted spectrum")
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend()

Note that the psf weighted extraction has visabily higher S/N, especially at the longer wavelengths where the S/N is lowest overall.

In [ ]:
# plot in Specviz
specviz2 = Specviz()
specviz2.app

In [ ]:
ext1d_boxcar_spec1d = Spectrum1D(spectral_axis=waves_boxcar[gpts]*u.micron, flux=ext1d_boxcar[gpts]*u.Jy)
ext1d_boxcar_bkgsub_spec1d = Spectrum1D(spectral_axis=waves_boxcar_bkgsub[gpts]*u.micron, flux=ext1d_boxcar_bkgsub[gpts]*u.Jy)
ext1d_psfweight_spec1d = Spectrum1D(spectral_axis=waves_boxcar_bkgsub_psfweight[gpts]*u.micron, flux=ext1d_boxcar_bkgsub_psfweight[gpts]*u.Jy)

In [ ]:
specviz2.load_spectrum(ext1d_boxcar_spec1d, data_label='boxcar')
specviz2.load_spectrum(ext1d_boxcar_bkgsub_spec1d, data_label='boxcar bkgsub')
specviz2.load_spectrum(ext1d_psfweight_spec1d, data_label='psfweight')

# 6) PSF-Fitted Extraction


#### Developer note:  Not currently possible. Allow for wavelength scaled width in the boxcar

# 7) Specviz2D Extraction

### Watch these two demo videos on how to extract your spectra using Specviz2D

In [ ]:
# Video showing how to use specviz2d
HTML('<iframe width="700" height="500" src="https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/videos/lrs_specviz2d.mov" frameborder="0" allowfullscreen></iframe>')

In [ ]:
# Video showing how to use specviz2d
HTML('<iframe width="700" height="500" src="https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/MIRI_LRS_notebook/videos/lrs_plotting.mov" frameborder="0" allowfullscreen></iframe>')

In [ ]:
specviz2d = Specviz2d()
specviz2d.app

In [ ]:
specviz2d.load_data(s2dfile)

In [ ]:
boxcar = specviz2d.app.get_data_from_viewer('spectrum-viewer',data_label='boxcar')
horne = specviz2d.app.get_data_from_viewer('spectrum-viewer',data_label='horne')

In [ ]:
# Check to see if user made a boxcar extraction, otherwise read in from file
if not boxcar:
    print("You didn't extract a spectrum from specviz2d, we will load a pre-extracted spectrum from the video above.")
    boxcar_specviz2d = Spectrum1D.read('boxcar_specviz2d.fits')
else:
    myboxcar = boxcar.flux.value
    myboxcar *= 1e6 * s2d.meta.photometry.pixelarea_steradians
    boxcar_specviz2d = Spectrum1D(spectral_axis=jpipe_x1d.spectral_axis, flux=myboxcar*u.Jy)

In [ ]:
if not horne:
    print("You didn't extract a spectrum from specviz2d, we will load a pre-extracted spectrum from the video above.")
    horne_specviz2d = Spectrum1D.read('horne_specviz2d.fits')
else:
    myhorne = horne.flux.value
    myhorne *= 1e6 * s2d.meta.photometry.pixelarea_steradians
    horne_specviz2d = Spectrum1D(spectral_axis=jpipe_x1d.spectral_axis, flux=myhorne*u.Jy)

In [ ]:
# Sarah's Extraction (for reference)
sp3_x1dfile = 'data/PID2072_Obs1_LRS_demo_x1d.fits'
sp3_x1d = datamodels.open(sp3_x1dfile)
ll3 = (sp3_x1dfile.split('/')[-1]).split('.')[0] + ' (Level 3, custom)'

In [ ]:
# plot
fig, ax = plt.subplots(figsize=(6, 6))
gpts = ext1d_boxcar_bkgsub > 0.
gpts = ext1d_boxcar > 0.

ax.plot(jpipe_x1d.spectral_axis, ext1d_boxcar_noweights, color = 'blue', label = 'Boxcar; No Mask')
ax.plot(jpipe_x1d.spectral_axis, ext1d_horne, color = 'purple', label = 'Horne; No Mask')
ax.plot(waves_boxcar[gpts], ext1d_boxcar[gpts], color = 'green', label="Boxcar; Mask")
ax.plot(waves_boxcar_bkgsub[gpts], ext1d_boxcar_bkgsub[gpts], color = 'red', label="Boxcar; bkgsub; Mask")
ax.plot(waves_boxcar_bkgsub_psfweight[gpts], ext1d_boxcar_bkgsub_psfweight[gpts], 'k-', label="Boxcar; bkgsub; PSF Weights", color='cyan')
ax.plot(jpipe_x1d.spectral_axis, jpipe_x1d.flux, color = 'orange', label="jpipe_x1d")

ax.plot(boxcar_specviz2d.spectral_axis, boxcar_specviz2d.flux, 'k-', label="Boxcar Specviz2D", color='magenta')
ax.plot(horne_specviz2d.spectral_axis, horne_specviz2d.flux, 'k-', label="Horne Specviz2D", color='lawngreen')
ax.plot(sp3_x1d.spec[0].spec_table['WAVELENGTH'], sp3_x1d.spec[0].spec_table['FLUX'], label=ll3, color = 'gold')
ax.set_title("Fixed boxcar 1D extracted spectrum")
ax.set_xlabel(r"wavelength [$\mu$m]")
ax.set_ylabel("Flux Density [Jy]")
ax.set_yscale("log")
ax.set_ylim(1e-6, 3e-3)
ax.set_xlim(4, 13)
ax.legend(bbox_to_anchor=(1.1, 1.05))

## Additional Resources

- [MIRI LRS](https://jwst-docs.stsci.edu/mid-infrared-instrument/miri-observing-modes/miri-low-resolution-spectroscopy)
- [MIRISim](http://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/mirisim)
- [JWST pipeline](https://jwst-docs.stsci.edu/jwst-data-reduction-pipeline)
- PSF weighted extraction [Horne 1986, PASP, 98, 609](https://ui.adsabs.harvard.edu/abs/1986PASP...98..609H/abstract).

## About this notebook

**Author:** Karl Gordon, JWST
**Updated On:** 2020-07-07

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 